In [2]:
%pylab inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


In [140]:
import devkit.api as dk
import pandas as pd
import numpy as np
from sklearn.decomposition import KernelPCA
from sklearn.svm import SVR
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.tree import DecisionTreeRegressor  # 导入决策树分类器
from sklearn.model_selection import GridSearchCV  # 导入网格搜索便于调试找到最优参数

In [105]:
ts_szzz1 = pd.read_csv(r"E:\99_daily\TODAY\szzz.csv")  # 上证综指2000-2018年前复权收盘价及其他ts指标

# 设置自变量和因变量
ts_szzz1['x1'] = np.nan
ts_szzz1['x2'] = np.nan
ts_szzz1['x3'] = np.nan
ts_szzz1['y'] = np.nan

id_x1 = ts_szzz1.columns.tolist().index('x1')
id_x2 = ts_szzz1.columns.tolist().index('x2')
id_x3 = ts_szzz1.columns.tolist().index('x3')
id_y = ts_szzz1.columns.tolist().index('y')
id_close = ts_szzz1.columns.tolist().index('close')


for i in range(close.shape[0]):
    if i > 2 and i < close.shape[0] - 1:
        ts_szzz1.iloc[i, id_x1] = ((ts_szzz1.iloc[i, id_close] / ts_szzz1.iloc[i - 1, id_close]) - 1) * 100
        ts_szzz1.iloc[i, id_x2] = ((ts_szzz1.iloc[i, id_close] / ts_szzz1.iloc[i - 2, id_close]) - 1) * 100
        ts_szzz1.iloc[i, id_x3] = ts_szzz1.iloc[i, id_close] - (ts_szzz1.iloc[i - 1, id_close] + ts_szzz1.iloc[i - 2, id_close] + ts_szzz1.iloc[i - 3, id_close]) / 3
        ts_szzz1.iloc[i, id_y] = 100 * (ts_szzz1.iloc[i + 1, id_close] / (ts_szzz1.iloc[i - 1, id_close] + ts_szzz1.iloc[i - 2, id_close] + ts_szzz1.iloc[i, id_close]) * 3 - 1)

ts_szzz2 = ts_szzz1.set_index('date')

train_pre1 = ts_szzz2[['x1', 'x2', 'x3', 'y']]
train_pre2 = train_pre1.dropna()
train_pre3 = train_pre2.apply(lambda x: (x - x.mean()) / x.std())  # 统计标准化
#  盖帽法处理极端值
def handle_outliers(x):
    x[np.abs(x)>3] = 3
    return x

train_pre4 = train_pre3.apply(lambda x: handle_outliers(x))

In [122]:
x_train = train_pre4.iloc[:3500,:3]
x_test = train_pre4.iloc[3500:,:3]
y_train = train_pre4.iloc[:3500,3]
y_test = train_pre4.iloc[3500:,3]

In [129]:
svr_rbf = SVR(C=70,epsilon=0.5)
svr_rbf.fit(x_train, y_train)
print("训练集R2= ", svr_rbf.score(x_train, y_train))
print("测试集R2= ", svr_rbf.score(x_test, y_test))

训练集R2=  0.36060204167
测试集R2=  0.161383804735


In [146]:
svr_rbf = SVR(C=0.45,epsilon=0.6)
svr_rbf.fit(x_train, y_train)
print("训练集R2= ", svr_rbf.score(x_train, y_train))
print("测试集R2= ", svr_rbf.score(x_test, y_test))

训练集R2=  0.324486470742
测试集R2=  0.232560846471


In [148]:
svr_rbf = SVR(C=0.251,epsilon=0.6)
svr_rbf.fit(x_train, y_train)
print("训练集R2= ", svr_rbf.score(x_train, y_train))
print("测试集R2= ", svr_rbf.score(x_test, y_test))

训练集R2=  0.320756556988
测试集R2=  0.233118299227


In [147]:
# 网格法寻优
# 首先对n_estimators（迭代次数）进行网格搜索
param_test1 = {'kernel': ['rbf'], 'C': np.arange(0.001,1,0.05), 'epsilon': np.arange(0,1,0.05)}
gsearch1 = GridSearchCV(estimator=SVR(), param_grid=param_test1, cv=5)
gsearch1.fit(train_pre4.iloc[:,:3], train_pre4.iloc[:,3])
print('最佳score值：', gsearch1.best_score_, '\n 最佳参数数', gsearch1.best_params_)

最佳score值： 0.304314940326 
 最佳参数数 {'C': 0.251, 'epsilon': 0.5, 'kernel': 'rbf'}


In [145]:
# 网格法寻优
# 首先对n_estimators（迭代次数）进行网格搜索
param_test1 = {'kernel': ['rbf'], 'C': np.arange(0.001,1,0.05), 'epsilon': np.arange(0,1,0.05)}
gsearch1 = GridSearchCV(estimator=SVR(), param_grid=param_test1, cv=5)
gsearch1.fit(x_train, y_train)
print('最佳score值：', gsearch1.best_score_, '\n 最佳参数数', gsearch1.best_params_)

最佳score值： 0.314853692846 
 最佳参数数 {'C': 0.45001000000000002, 'epsilon': 0.60000000000000009, 'kernel': 'rbf'}


In [138]:
param_test1 = {'kernel': ['rbf','poly','sigmoid'], 'C': np.arange(0,1,0.05), 'epsilon': np.arange(0,1,0.05)}

In [137]:
np.arange(0,1,0.05)

array([ 0.  ,  0.05,  0.1 ,  0.15,  0.2 ,  0.25,  0.3 ,  0.35,  0.4 ,
        0.45,  0.5 ,  0.55,  0.6 ,  0.65,  0.7 ,  0.75,  0.8 ,  0.85,
        0.9 ,  0.95])

In [131]:
tree_regressor = DecisionTreeRegressor()
tree_regressor.fit(x_train, y_train)
print("训练集R2= ", tree_regressor.score(x_train, y_train))
print("测试集R2= ", tree_regressor.score(x_test, y_test))

训练集R2=  0.980479103026
测试集R2=  -0.504480009426


In [132]:
clf = linear_model.Lasso(alpha=0.4)
clf.fit(x_train, y_train)
print("训练集R2= ", clf.score(x_train, y_train))
print("测试集R2= ", clf.score(x_test, y_test))

训练集R2=  0.0510871745065
测试集R2=  0.0317275656623


SVR(C=100, cache_size=200, coef0=0.0, degree=3, epsilon=0.001, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [91]:
print("训练集R2= ", svr_rbf.score(x_train, y_train))
print("测试集R2= ", svr_rbf.score(x_test, y_test))

训练集R2=  0.358913163286
测试集R2=  0.181610869678


In [97]:
print("训练集R2= ", svr_rbf.score(x_train, y_train))
print("测试集R2= ", svr_rbf.score(x_test, y_test))

训练集R2=  0.34330815152
测试集R2=  0.335216904019
